# Arcane Python syntax

Python has a syntax with a few features that are quite unique.

## Interval checking

In Python an expression such as `a < x <= b` is legal and well-defined.

In [1]:
a, b = 1, 3
for x in range(-2, 5):
    if a < x <= b:
        print(f'{a} < {x} <= {b}: True')
    else:
        print(f'{a} < {x} <= {b}: False')

1 < -2 <= 3: False
1 < -1 <= 3: False
1 < 0 <= 3: False
1 < 1 <= 3: False
1 < 2 <= 3: True
1 < 3 <= 3: True
1 < 4 <= 3: False


The code above can be simplified to:

In [3]:
a, b = 1, 3
for x in range(-2, 5):
    print(f'{a} < {x:2d} <= {b}: {a < x <= b}')

1 < -2 <= 3: False
1 < -1 <= 3: False
1 <  0 <= 3: False
1 <  1 <= 3: False
1 <  2 <= 3: True
1 <  3 <= 3: True
1 <  4 <= 3: False


## Multiple equality, inequality

Along the same lines, `a == b == x` is also legal and well-defined.

In [4]:
for a in range(3):
    for b in range(3):
        print(f'{a} == {b} == 1: {a == b == 1}')

0 == 0 == 1: False
0 == 1 == 1: False
0 == 2 == 1: False
1 == 0 == 1: False
1 == 1 == 1: True
1 == 2 == 1: False
2 == 0 == 1: False
2 == 1 == 1: False
2 == 2 == 1: False


Although `a != b != x` is legal as well, it may, at least at first sight, not behave as expected.

In [5]:
for a in range(3):
    for b in range(3):
        print(f'{a} != {b} != 1: {a != b != 1}')

0 != 0 != 1: False
0 != 1 != 1: False
0 != 2 != 1: True
1 != 0 != 1: True
1 != 1 != 1: False
1 != 2 != 1: True
2 != 0 != 1: True
2 != 1 != 1: False
2 != 2 != 1: False


From the above, it is clear that `a != b != x` translates to `a != b and b != c`, which is true when `a == c and a != b`.

## Iteration with `else`

Iteration statements in Python, i.e., `for` and `while` can have an `else` block.  The latter is executed when the iteration statement terminates normally, i.e., not by a `bread` statement.

In [6]:
def illustrate_for_else(x):
    for i in range(10):
        if i == x:
            print('break')
            break
    else:
        print('no break')

In [7]:
illustrate_for_else(12)

no break


In [8]:
illustrate_for_else(5)

break


Although naming this syntactic construct `else` feels awkward, it is quite useful, since it is a syntactic shortcut for the following reasonably common construct.

In [12]:
def illustrate_for_bool(x):
    completed_succesfully = True
    for i in range(10):
        if i == x:
            print('break')
            completed_succesfully = False
            break
    if completed_succesfully:
        print('no break')

In [13]:
illustrate_for_bool(12)

no break


In [14]:
illustrate_for_bool(5)

break


The execution of `continue` has no influence on this.

In [10]:
for i in range(5):
    if i > -1:
        continue
    print(f'did something for {i}')
else:
    print('completed normally')

completed normally


The `while` statement can have an `else` with the same semantics.